In [1]:
import numpy as np
import requests
import pandas as pd
import xlsxwriter
import math
from secret import IEX_CLOUD_API_TOKEN

In [2]:
# stocks = pd.read_csv('sp_500_stocks.csv')

In [3]:
# stocks

In [4]:
# symbol = 'AAPL'
api_url = 'https://cloud.iexapis.com//stable'
# stock_data_url = f'{api_url}/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
# data = requests.get(stock_data_url).json()
# data



In [5]:
# price = data['latestPrice']
# market_cap = data['marketCap']
# price

In [6]:
# Symbol = []
# Price = []
# Market_Cap = []
# for x in range(len(stocks['Ticker'])):
#     symbol = stocks['Ticker'][x]
#     stock_data_url = f'{api_url}/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
#     try:
#         data = requests.get(stock_data_url).json()
#     except:
#         continue
#     else:
#         price = data['latestPrice']
#         market_cap = data['marketCap']
#         Symbol.append(symbol)
#         Price.append(price)
#         Market_Cap.append(market_cap)
#         print(Symbol)
#         print(Market_Cap)
#         print(Price)

In [7]:
# full_data2 = {
#     'Ticker': Symbol,
#     'Price': Price,
#     'Market Cap': Market_Cap
#             }
# Data2 = pd.DataFrame(full_data)
# Data2.tail()

In [8]:
# Data2.to_csv('sp_500_stocksdata.csv', index=False)

In [9]:
# Data2['No of Shares to Buy'] = 'N/A'

Data2 = pd.read_csv('sp_500_stocksdata.csv')
Symbol = Data2['Ticker']

In [10]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i+n]

In [11]:
chunks(Symbol, 100)

<generator object chunks at 0x000001BC159A90E0>

In [12]:
Tickers = list(chunks(Symbol, 100))
my_columns = ['Ticker', 'Price', 'Market Capitalization', 'Number of Shares to Buy']
symbol_strings = []
for i in range (0, len(Tickers)):
    symbol_strings.append(','.join(Tickers[i]))
df  = pd.DataFrame(columns=my_columns)
for symbol_string in symbol_strings:
    # print(symbol_string)
    api_url = 'https://cloud.iexapis.com//stable'
    batch_api_call_url = f"{api_url}/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}"
    # print(batch_api_call_url)
    data = requests.get(batch_api_call_url).json()
    # print(data)
    for symbol in symbol_string.split(','):
        df = df.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['quote']['latestPrice'],
                    data[symbol]['quote']['marketCap'],
                    'N/A'
                ], index=my_columns
            ), ignore_index=True
        )
print(df)

C:\Users\panka\AppData\Local\Temp\ipykernel_13444\542703600.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
C:\Users\panka\AppData\Local\Temp\ipykernel_13444\542703600.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
C:\Users\panka\AppData\Local\Temp\ipykernel_13444\542703600.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
C:\Users\panka\AppData\Local\Temp\ipykernel_13444\542703600.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
C:\Users\panka\AppData\Local\Temp\ipykernel_13444\542703600.py:15: FutureWarning: The frame.append method is deprecated and will

    Ticker   Price Market Capitalization Number of Shares to Buy
0        A  138.52           40960620678                     N/A
1      AAL   14.32            9348332151                     N/A
2      AAP  122.18            7242070563                     N/A
3     AAPL  160.80         2544168916800                     N/A
4     ABBV  161.38          284697201586                     N/A
..     ...     ...                   ...                     ...
496    YUM  131.57           36853791535                     N/A
497    ZBH  131.38           27455826033                     N/A
498   ZBRA  304.03           15626941644                     N/A
499   ZION   30.11            4459262636                     N/A
500    ZTS  171.21           79194708413                     N/A

[501 rows x 4 columns]


C:\Users\panka\AppData\Local\Temp\ipykernel_13444\542703600.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(


In [25]:
portfolio_size = input('Enter the value of your portfolio ')
while (portfolio_size.isnumeric() != True):
    print('Please enter a valid input ')
    portfolio_size = input('Enter the value of your portfolio ')
val = float(portfolio_size)

In [36]:
position_size = val/len(df.index)
no_of_shares_to_buy = position_size/df['Price']
print(no_of_shares_to_buy)
df['Number of Shares to Buy'] = no_of_shares_to_buy
df
# position_size

0       28.819058
1      278.772065
2       32.673236
3       24.825970
4       24.736745
          ...    
496     30.341385
497     30.385264
498     13.130336
499    132.581068
500     23.316488
Name: Price, Length: 501, dtype: float64


,Ticker,Price,Market Capitalization,Number of Shares to Buy
0,A,138.52,40960620678,28.819058
1,AAL,14.32,9348332151,278.772065
2,AAP,122.18,7242070563,32.673236
3,AAPL,160.80,2544168916800,24.825970
4,ABBV,161.38,284697201586,24.736745
...,...,...,...,...
496,YUM,131.57,36853791535,30.341385
497,ZBH,131.38,27455826033,30.385264
498,ZBRA,304.03,15626941644,13.130336
499,ZION,30.11,4459262636,132.581068


In [39]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')
df.to_excel(writer, 'recommended_trades', index=False)


In [41]:
background_colour = '#0a0a23'
font_colour = '#fffff'
string_format = writer.book.add_format(
    {
        'font_color': font_colour,
        'bg_color': background_colour,
        'border': 1
    }
)
dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_colour,
        'bg_color': background_colour,
        'border': 1
    }
)
integer_format = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_colour,
        'bg_color': background_colour,
        'border': 1
    }
)



In [43]:
writer.sheets['recommended_trades'].set_column('A:A', 18, string_format)
writer.save()